In [19]:
"""
pip install opencv-python  # required for cv2
"""

import cv2
import numpy as np
from pathlib import Path

def collate_images(background_path, image_paths, output_dir=Path("output"), scale_factor=0.8, suffix="bgd"):
    """
    Collates images with a background, preserving aspect ratio and using a fixed scale factor.

    Args:
        background_path: Path to the background image.
        image_paths: List of paths to images to be collated.
        output_dir: Output directory Path for the collated images.
        scale_factor: The scaling factor for the images (0.0 - 1.0).
        suffix: added to the new output filename
    """
    # Ensure output directory exists
    output_dir.mkdir(parents=True, exist_ok=True)
    
    try:
        background = cv2.imread(str(background_path))
    except cv2.error as e:
        print(f"Error reading background image: {e}")
        return False

    bg_height, bg_width, _ = background.shape

    for image_path in image_paths:
        try:
            image = cv2.imread(str(image_path))
        except cv2.error as e:
            print(f"Error reading image: {image_path}, {e}")
            continue  # Skip to the next image            

        if image is None: continue
            
        img_height, img_width, _ = image.shape

        # Determine the larger dimension and calculate new size based on scale factor
        if img_width > img_height:
            new_width = int(bg_width * scale_factor)
            new_height = int(img_height * (new_width / img_width))
        else:
            new_height = int(bg_height * scale_factor)
            new_width = int(img_width * (new_height / img_height))

        # Resize image
        resized_image = cv2.resize(image, (new_width, new_height))

        # Create a mask for the image
        mask = np.zeros_like(resized_image)
        mask[0:new_height, 0:new_width] = 255

        # Calculate position for placing the image on the background
        x = (bg_width - new_width) // 2
        y = (bg_height - new_height) // 2

        # Create a copy of the background image
        result = background.copy()

        # Put the resized image on the background
        result[y:y+new_height, x:x+new_width] = resized_image

        # Save the resulting image
        output_path = output_dir / (Path(image_path).stem + f"-{suffix}" + Path(image_path).suffix)
        cv2.imwrite(str(output_path), result)

if True:
    background_path = "images/00-background-stary-universe.png"
    image_paths = ["images/01-2-td-lee.jpeg", "images/01-100-td-lee.jpeg",]
    collate_images(Path(background_path), image_paths, output_dir=Path("new_images"))